In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -f https://release.oneflow.info oneflow==0.5.0+cu102

In [ ]:
!pip install /content/drive/MyDrive/Competition/oneflow-0.5.0+cu102-cp37-cp37m-manylinux2014_x86_64.whl

Processing ./drive/MyDrive/Competition/oneflow-0.5.0+cu102-cp37-cp37m-manylinux2014_x86_64.whl


In [ ]:
import oneflow as flow
import oneflow.nn as nn
import oneflow.nn.functional as F
import oneflow.utils.vision.datasets as dset
import oneflow.utils.vision.transforms as transforms
from oneflow.utils.data import DataLoader

import numpy as np

In [ ]:
DEVICE = "cuda" if flow.cuda.is_available() else "cpu"
print("Using {} device".format(DEVICE))

Using cuda device


In [ ]:
train_transform = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.RandomCrop(32, padding=4),
                                      #transforms.RandomRotation(degerees = 40), # no attribute 'RandomRotation'
                                      transforms.ToTensor()
                                      ])
test_transform = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.ToTensor()
                                      ])

In [ ]:
train = dset.ImageFolder('/content/drive/MyDrive/Competition/dataset/train', transform = train_transform)

In [ ]:
# define LeNet module
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Conv2d的第一个参数是输入的channel数量，第二个是输出的channel数量，第三个是kernel size
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),)
        
        self.flatten = nn.Flatten()

        # 最终有10类，所以最后一个全连接层输出数量是10
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),)

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)

        return x

In [ ]:
learning_rate = 0.008

model = LeNet().to("cuda")

criterion = nn.CrossEntropyLoss().to("cuda")
optimizer = flow.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [ ]:
trainloader = DataLoader(train, batch_size=16, shuffle=True)

print("Start Training...")
for epoch in range(50):

    # 我们用一个变量来记录每100个batch的平均loss
    loss100 = 0.0

    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to("cuda"), labels.to("cuda") # 注意需要复制到GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss100 += loss.item()
        # print('[Epoch %d, Batch %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
        if i % 100 == 99:
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss100 / 100))
            loss100 = 0.0

print("Done Training!")

In [ ]:
inputs.data.shape

In [ ]:
flow.save(model.state_dict(), "/content/drive/MyDrive/Competition/model_batch16_RandomCrop")

In [ ]:
model.load_state_dict(flow.load("/content/drive/MyDrive/Competition/model_batch16_RandomCrop"))

<All keys matched successfully>

In [ ]:
len(test.samples)

324

In [ ]:
test = dset.ImageFolder('/content/drive/MyDrive/Competition/dataset/test', transform = test_transform)
testloader = DataLoader(test, batch_size=16, shuffle=False)

predicted_list = []

# 使用torch.no_grad的话在前向传播中不记录梯度，节省内存
with flow.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to("cuda"), labels.to("cuda")
        # 预测
        outputs = model(inputs)
        # 我们的网络输出的实际上是个概率分布，取最大概率的哪一项作为预测分类
        _ , predicted = flow.max(outputs.data, 1)
        
        predicted_list.extend(list(predicted.numpy()))

In [ ]:
import json

json_list = []
for i in range(len(test.samples)):

  json_list.append({"filename": test.samples[i][0][43:], "label":str(predicted_list[i])})



f2 = open('test.json', 'w')
f2.write(json.dumps({"annotations": json_list}))
f2.close()

{'annotations': [{'filename': 'test/GuideSign/00004.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00053.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00055.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00058.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00074.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00079.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00117.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00172.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00175.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00183.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00206.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00216.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00220.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00252.jpg', 'label': 4},
  {'filename': 'test/GuideSign/00256.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00273.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00342.jpg', 'label': 0},
  {'filename': 'test/GuideSign/00